# 2. Spark Structured Streaming 설정
SparkSession을 생성하고 소켓으로 데이터를 수신하도록 스트리밍 설정을 합니다.

## 기본 스트리밍 설정

### TODO: 주석과 참고 페이지를 참고하여 실시간 스트리밍 데이터 처리를 가능하도록 완성해주세요.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split


# SparkSession 생성
# 스트리밍 애플리케이션을 실행하기 위한 기본 진입점으로, 세션을 생성하여 Spark 기능을 사용할 수 있게 설정합니다.
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.SparkSession.html
spark = SparkSession.builder.appName("StreamingWordCount").getOrCreate()

In [ ]:
# 소켓을 통한 스트리밍 데이터 소스 설정
# localhost의 9999 포트에서 데이터를 수신하여 DataFrame으로 생성합니다.
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameReader.format.html
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.SparkSession.readStream.html
lines = spark.______________ \
    .______________("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

In [ ]:
# 텍스트 라인을 단어로 분리
# 각 입력 줄(line)을 공백(" ")을 기준으로 분리하고, 개별 단어로 변환하여 DataFrame에 추가합니다.
# explode 함수로 각 단어를 개별 행으로 확장하고, alias로 "word" 컬럼명을 설정합니다.
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.explode.html
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.split.html
words = lines.select(______________(split(lines.value, " ")).alias("word"))

In [ ]:
# 각 단어의 출현 빈도 계산
# word 컬럼으로 그룹화하여 각 단어의 등장 횟수를 실시간으로 집계합니다.
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.groupBy.html
wordCounts = words.______________("word").count()

### Output Mode 설정
* Complete: 모든 집계 결과를 출력
* Append: 추가된 데이터만 출력
* Update: 변경된 데이터만 갱신하여 출력

In [ ]:
# 스트리밍 쿼리를 설정하여 실시간 결과를 출력
# outputMode("update"): 변경된 데이터만 갱신하여 출력합니다.
# format("console"): 결과를 콘솔에 출력하도록 설정합니다.
# option("truncate", "false"): 출력 결과를 잘리지 않고 전체 내용을 보여줍니다.
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.streaming.DataStreamWriter.outputMode.html
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.writeStream.html
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.streaming.DataStreamWriter.trigger.html
query = wordCounts.______________ \
    .______________("update") \
    .______________("console") \
    .option("truncate", "false") \
    .start()

# 스트리밍 쿼리를 계속 실행하여 데이터 수신을 대기합니다.
# 쿼리 실행 후 프로그램이 종료되지 않도록 유지합니다.
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.streaming.StreamingQuery.awaitTermination.html
query.awaitTermination()

In [ ]:
spark.stop()